In [274]:
import os
import yaml
import time
from src.utils import batch_generate, tokens_generate, run_inference
import importlib
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from dotenv import load_dotenv
import plotly.express as px
import torch
import torch.nn.functional as F
import huggingface
from datetime import datetime
load_dotenv('secrets.env')

dataset = config['datasets']['big_bench']
model_name = config['models']['deepseek_r1_qwendistill_1.5']

with open("config/config.yaml", "r") as f: 
    config = yaml.safe_load(f)

if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

In [273]:
dataset = load_dataset(dataset['source'], dataset['subset']).shuffle(config['seed'])
tokenizer = AutoTokenizer.from_pretrained(model_name) 
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

# Experiment
Goal: test model performance on 100 BBH questions w/ chain of thought reasoning =D

In [214]:
df = pd.DataFrame(dataset[0:100])

batches = batch_generate(df)
tokens = tokens_generate(batches, tokenizer, device = 'mps')
res = run_inference(model, tokens, tokenizer, time_tracking = True)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


KeyboardInterrupt: 